# Name: Mohammad Javad Noroozi
# Student Number: 99102434
# HW3

### import the necessary library

In [1]:
import json
import sys
import random
import datetime
from collections import Counter

In [2]:
import findspark
findspark.find()
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("HW1") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext


24/02/01 23:23:16 WARN Utils: Your hostname, javad-IdeaPad-Gaming-3-15IAH7 resolves to a loopback address: 127.0.1.1; using 192.168.137.8 instead (on interface enp48s0)
24/02/01 23:23:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/01 23:23:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read data

In [3]:
tweets_rdd = sc.textFile("twitter_data_v2.jsonl")


### Downsampling (If your RAM memory is low)

In [4]:
tweets_rdd = tweets_rdd.sample(False, 0.1, seed=42)


### Show a sample of tweets

In [5]:
# tweets_rdd.takeSample(False, 1, 19)


### Show size of data (No. tweets)

In [6]:
# tweets_rdd.count()


In [7]:
users = list(set(tweets_rdd.map(lambda x: json.loads(x)["user"].get('id')).collect()))

### Define Parameters for the Primary Rating.

In [8]:
# Define Parameters
owner_s = 1

# following parameters should not be equal to owner_s
rep_to_s = 0.2
rep_from_s = 0.9

ret_to_s = 0.5
ret_from_s = 1.5

qut_to_s = 0.6
qut_from_s = 1.5


### Define functions for primary and secondary ratings

In [9]:
def to_user_tid_score(tid, uid, type, origin_tid, origin_uid):
    ''' This function used for calculate primary rating matrix.
    
    This function get some information of a tweet and Returns
    the score that two users (the user who posted this tweet
    and the user who posted the original tweet and has now
    been replied to, retweeted, or quoted by this tweet)
    give to these two tweets FOR EACH USER.

    The output is a list of tuples, where each tuple contains
    a user ID and a list of tweets that have been rated (along
    with the value of the rating).
    Output format: [((u_i, ((t_j, s_ij),)) ...]
    '''
    origin_tid = [i for i in origin_tid if i is not None]
    origin_uid = [i for i in origin_uid if i is not None]

    if not origin_tid or not origin_uid:
        return [(uid, [(tid, owner_s)])]

    to_s = rep_to_s if type == "replied" else\
           ret_to_s if type == "retweeted" else\
           qut_to_s if type == "quoted" else\
           0
    from_s = rep_from_s if type == "replied" else\
             ret_from_s if type == "retweeted" else\
             qut_from_s if type == "quoted" else\
             0

    return [(uid, [(tid, owner_s), (origin_tid[0], from_s)]),
            (origin_uid[0], [(tid, to_s)]),
           ]


def to_tid_user_score(tid, uid, type, origin_tid, origin_uid):
    ''' This function used for secondary rating.

    This function get some information of a tweet and Returns
    the score that two users (the user who posted this tweet
    and the user who posted the original tweet and has now
    been replied to, retweeted, or quoted by this tweet)
    give to these two tweets FOR EACH TWEET.

    The output is a list of tuples, where each tuple contains
    a tweet ID and a tuple list of users rate to it (along with
    the value of the rating).
    Output format: [(u_i, ((t_j, s_ij), ...)), ...]
    '''
    origin_tid = [i for i in origin_tid if i is not None]
    origin_uid = [i for i in origin_uid if i is not None]

    if not origin_tid or not origin_uid:
        return [(tid, ((uid, owner_s),))]

    to_s = rep_to_s if type == "replied" else\
           ret_to_s if type == "retweeted" else\
           qut_to_s if type == "quoted" else\
           0
    from_s = rep_from_s if type == "replied" else\
             ret_from_s if type == "retweeted" else\
             qut_from_s if type == "quoted" else\
             0

    return [(tid, ((uid, owner_s), (origin_uid[0], to_s),)),
            (origin_tid[0], ((uid, from_s),)),
           ]


def extract_tuples(tweet_info, func):
    '''
    This function extract some information of tweet and send it
    to one of the following funcions and return their results:
    - to_user_tid_score
    - to_tid_user_score
    '''
    return func(
        tweet_info.get('id'),
        tweet_info["user"].get('id'),
        tweet_info.get('tweet_type'),
        [
            tweet_info.get('in_reply_to_status_id_str', None),
            tweet_info.get('quoted_status', {}).get('id'),
            tweet_info.get('retweeted_status', {}).get('id'),
        ],
        [
            tweet_info.get('in_reply_to_user_id_str', None),
            tweet_info.get('quoted_status', {}).get('user', {}).get('id'),
            tweet_info.get('retweeted_status', {}).get('user', {}).get('id'),
        ],
    )


def extract_relations2(x):
    '''
    This function extracts the MAX_REALTION strongest relationships
    between users who interacted with this tweet.
    
    x contains the tweet_id (tid) and a list of tuples. Each tuple
    contains a user_id (u) and its interaction with the tweet (score
    given to the tweet).
    x: (tid, [(u1, s1), ...])

    The output is a list that each element of it, contains the user_id
    (u1) and a list of tuples. Each tuple contains a tweet_id (tid), 
    second user_id that has relation to u1 (called u2), and the score
    given by each of these two users to this tweet (s1 and s2 respectively)
    output: 
    [(u1, [(tid, u2, s1, s2),
            (tid, u3, s1, s3),
            ...
           ]),
     (u2, [(tid, u1, s2, s1),
            (tid, u3, s2, s3),
            ...
                  ]),
     ...
    ]
    
    '''
    MAX_REALTION = 10

    tid, user_score = x
    relations = []
    if len(user_score) > MAX_REALTION:
        user_score = [(u1, s1) for (u1, s1) in user_score if s1 > rep_to_s]
    if len(user_score) > MAX_REALTION:
        user_score = [(u1, s1) for (u1, s1) in user_score if s1 > ret_to_s]


    for (u1, s1) in user_score:
        u1_relations = []
        for (u2, s2) in user_score:
            if u1 != u2 and s1 > rep_to_s:
                u1_relations.append((tid, u2, s1, s2))

        # if len(u1_relations) > MAX_REALTION:
        #     relations.append(
        #         (u1, sorted(u1_relations, key=lambda x: -x[2]*x[3])[:10]))
        # else:
        relations.append((u1, u1_relations))
        
    return relations



def extract_relations(x):
    '''
    This function extracts the MAX_REALTION strongest relationships
    between users who interacted with this tweet.
    
    x contains the tweet_id (tid) and a list of tuples. Each tuple
    contains a user_id (u) and its interaction with the tweet (score
    given to the tweet).
    x: (tid, [(u1, s1), ...])

    The output is a list that each element of it, contains the user_id
    (u1) and a list of tuples. Each tuple contains a tweet_id (tid), 
    second user_id that has relation to u1 (called u2), and the score
    given by each of these two users to this tweet (s1 and s2 respectively)
    output: 
    [
        (u1,   (
                    ((tid, s1), [(u2, s2),
                                (u3, s3),
                                ...
                                ]
                    ),
                )
        ),
        (u2,   (
                    ((tid, s2), [(u1, s1),
                                (u3, s3),
                                ...
                                ]
                    ),
                )
        ),
     ...
    ]
    
    '''
    # MAX_REALTION = 10


    # if len(user_score) > MAX_REALTION:
    #     user_score = [(u1, s1) for (u1, s1) in user_score if s1 > rep_to_s]
    # if len(user_score) > MAX_REALTION:
    #     user_score = [(u1, s1) for (u1, s1) in user_score if s1 > ret_to_s]

    tid, user_score = x
    relations = []

    for (u1, s1) in user_score:
        if s1 > rep_to_s:
            u1_relations = []
            for (u2, s2) in user_score:
                if u1 != u2:
                    u1_relations.append((u2, s2))

            relations.append((u1, (((tid, s1), u1_relations), )))
        
    return relations


def select_best(x):
    '''
    This function select 25 strongest relations from relations 
    exist in x.
    x contains the user_id (u1) and a list of tuples. Each tuple
    contains a tweet_id (tid), second user_id that has relation 
    to u1 (called u2), and the score given by each of these two
    users to this tweet (s1 and s2 respectively)

    x: (u1, ( ((tid, s1),   [(uj,sj),]), ...))
    '''
    u1 = x[0]
    tid_in_relation_with_score, users_in_relation = zip(*x[1])
    # u1, relations = x
    if relations:
        th = rep_to_s
        while len(relations) > 25:
            new_relations = [r for r in relations if r[2]*r[3] > th]
            if len(new_relations) < 15:
                break
            relations = new_relations
            th *= 1.1

    return (u1, relations)


def find_user_interests(x):
    '''
    x:  (u1,   (
                    ((tid, s1), [(u2, s2),
                                (u3, s3),
                                ...
                                ]
                    ),
                )
        ),

    note: s1 > rep_to_s

    x: (u1, ( ((tid, s1),   [(uj,sj),]), ...))
    tid_in_relation_with_score: ((tid_i, s1_i), ...)
    users_in_relation: ([(uj, sj), ...], [(uk, sk), ...], ...)
    '''
        # x: (u1, ( ((tid, s1),   [(uj,sj),]), ...))

    u1 = x[0]

    tid_in_relation_with_score, users_in_relation = zip(*x[1])
    interest = []
    for i, (tid, s1_i) in enumerate(tid_in_relation_with_score):
        if s1_i != owner_s:
            interest.append((u1, (tid, s1_i)))
        if s1_i > rep_to_s:
            for (uj, sj) in users_in_relation[i]:
                for k, (tid2, s1_k) in enumerate(tid_in_relation_with_score):
                    if i != k and sj != owner_s:
                        interest.append((uj, {tid2: s1_i*sj*s1_k}))

    return interest

def find_user_interests3(x):
    '''
    x:  (u1,   (
                    ((tid), [(u2),
                            (u3),
                            ...
                            ]
                    ),
                )
        ),
        
        x: (u1, ( (tid,   [uj, ]), ...))

        tid_in_relation: ((tid_i), ...)
        users_in_relation: ([(uj), ...], [(uk, sk), ...], ...)
    '''
        # x: (u1, ( ((tid, s1),   [(uj,sj),]), ...))

    u1 = x[0]

    tid_in_relation_with_score, users_in_relation = zip(*x[1])
    interest = {}
    for i, (tid, s1_i) in enumerate(tid_in_relation_with_score):
        if s1_i > rep_to_s:
            for (uj, sj) in users_in_relation[i]:
                for k, (tid2, s1_k) in enumerate(tid_in_relation_with_score):
                    if i != k:
                        interest[uj] = interest.get(uj, set()).union({tid2})


def find_user_interests2(x, user_scores):
    ''' 
        user_scores: {ui: [(t_j, s_ij), ...]}
        x: (u1, ((tid, uj, s1, sj), ...))
    '''
    th = 0.05
    u, relations = x
    interest = {}
    if relations:
        for (tid, uj, s1, s2) in relations:
            interest[tid] = interest.get(tid, 0) + s1
            for (t, s) in user_scores.get(uj, []):
                interest[t] = interest.get(t, 0) + s1*s2*s

    if interest:
        max_interest = max(interest.values())
        th = 0.05 * max_interest
        while len(interest) > 100:
            interest = {t:i for t,i in interest.items() if i > th}
            th *= 1.1
        return (u, interest)
    return (u,{})


In [10]:
startt = datetime.datetime.now()
print(startt)


2024-02-01 23:23:34.824573


In [11]:
# # [((u_i, ((t_j, s_ij),)), ...]
# user_tid_score_rdd = tweets_rdd.flatMap(
#     lambda x: extract_tuples(json.loads(x), func=to_user_tid_score))

# # [(u_i, ((t_j, s_ij), ...)), ...]
# user_tids_scores_rdd = user_tid_score_rdd.reduceByKey(
#     lambda x, y: x if not x.extend(y) else x)

# user_scores_list = user_tids_scores_rdd.collect()

# user_scores_dict = dict(user_scores_list)


In [12]:
# # [(tid, [(u, s)]), ...]
# tid_user_score_rdd = tweets_rdd.flatMap(lambda x: extract_tuples(json.loads(x), func=to_tid_user_score))

# # [(tid, [(u, s), ...]), ...]
# tid_users_scores_rdd = tid_user_score_rdd.reduceByKey(lambda x, y: x + y)

# # [(u1, [(tid, u2, s1, s2)])]
# user_relation_rdd = tid_users_scores_rdd.flatMap(extract_relations)

# # [(u1, [(tid, u2, s1, s2), ...])]
# user_relations_rdd = user_relation_rdd.reduceByKey(lambda x, y: x if not x.extend(y) else x)\
#                                       .filter(lambda x:x[1])

# # [(u1, [(tid, u2, s1, s2), ...])]
# user_best_relations_rdd = user_relations_rdd.map(select_best)

# [(u1, [(tid: interest), ...])]
# user_interests_rdd = user_best_relations_rdd.map(lambda x: find_user_interests(x, user_scores_dict))

# selected_user_interests_rdd = user_interests_rdd.filter(lambda x: x[0] == '933731737686994946')


In [13]:
# [(tid, [(u, s)]), ...]
tid_user_score_rdd = tweets_rdd.flatMap(lambda x: extract_tuples(json.loads(x), func=to_tid_user_score))

# [(tid, [(u, s), ...]), ...]
tid_users_scores_rdd = tid_user_score_rdd.reduceByKey(lambda x, y: x + y)

# [(u1, (((tid, s1),   [(uj,sj),]),))]
user_relation_rdd = tid_users_scores_rdd.flatMap(extract_relations)\
                                      .filter(lambda x:x[1][0][1])

# [(u1, (((tid, s1), [(uj,sj),]), ...))]
user_relations_rdd = user_relation_rdd.reduceByKey(lambda x, y: x + y)

# [(uj, {tid: interest})]
user_interests_flat_rdd = user_relations_rdd.flatMap(lambda x: find_user_interests(x))

# [(uj, {tid: interest})]
user_interests_rdd = user_interests_flat_rdd.reduceByKey(lambda x, y: dict(Counter(x) + Counter(y)))

# selected_user_interests_rdd = user_interests_rdd.filter(lambda x: x[0] == '933731737686994946')


In [15]:
'''
you must choose user id in this section.
you can change # in `list(user_scores_dict.keys())[#]` for this purpose.
'''
# uid = list(user_scores_dict.keys())[1]
uid = users[1]
selected_user_interests = user_interests_rdd.filter(lambda x: x[0] == uid).take(1)


In [16]:
necessary_keys = ['tweet_type', 'user', 'id','text', 'name', 'screen_name', 
                  'in_reply_to_status_id_str', 'in_reply_to_user_id_str',
                  'quoted_status', 'retweeted_status','profile_image_url',
                 ]


In [ ]:
interests_tweets = tweets_rdd.filter(lambda x: json.loads(x)['id'] in selected_user_interests[0][1].keys() or
                                               json.loads(x)['user']['id'] == uid)\
                             .map(lambda x: dict(filter(lambda y: y[0] in necessary_keys, json.loads(x).items())))\
                             .collect()
interests_tweets2 = interests_tweets.copy()


In [ ]:
def extract_user_origin_tweets(tweets, uid, rem=True):
    origin_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'generated':
            if t.get('user',{}).get('id','0') == uid:
                origin_tweets.append(t)
    if rem:
        for t in origin_tweets:
            tweets.remove(t)
    return origin_tweets


def extract_user_replied_tweets(tweets, uid, rem=True):
    replied_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'replied':
            if t.get('user',{}).get('id','0') == uid:
                replied_tweets.append(t)
    if rem:
        for t in replied_tweets:
            tweets.remove(t)
    return replied_tweets


def extract_user_quoted_tweets(tweets, uid, rem=True):
    quoted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'quoted':
            if t.get('user',{}).get('id','0') == uid:
                quoted_tweets.append(t)
    if rem:
        for t in quoted_tweets:
            tweets.remove(t)
    return quoted_tweets


def extract_user_retweeted_tweets(tweets, uid, rem=True):
    retweeted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'retweeted':
            if t.get('user',{}).get('id','0') == uid:
                retweeted_tweets.append(t)
    if rem:
        for t in retweeted_tweets:
            tweets.remove(t)
    return retweeted_tweets


def extract_replied_to_user_tweets(tweets, uid, rem=True):
    replied_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'replied':
            if t.get('in_reply_to_user_id_str', '0') == uid:
                replied_tweets.append(t)
    if rem:
        for t in replied_tweets:
            tweets.remove(t)
    return replied_tweets


def extract_quoted_user_tweets(tweets, uid, rem=True):
    quoted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'quoted':
            if t.get('quoted_status', {}).get('user', {}).get('id', '0') == uid:
                quoted_tweets.append(t)
    if rem:
        for t in quoted_tweets:
            tweets.remove(t)
    return quoted_tweets


def extract_retweeted_user_tweets(tweets, uid, rem=True):
    retweeted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'retweeted':
            if t.get('retweeted_status', {}).get('user', {}).get('id', '0') == uid:
                retweeted_tweets.append(t)
    if rem:
        for t in retweeted_tweets:
            tweets.remove(t)
    return retweeted_tweets


def extract_explore_tweets(tweets):
    return tweets


In [ ]:
interests_tweets = interests_tweets2.copy()
user_origin_tweets = extract_user_origin_tweets(interests_tweets, uid)
user_quoted_tweets  = extract_user_quoted_tweets(interests_tweets, uid)
user_retweeted_tweets  = extract_user_retweeted_tweets(interests_tweets, uid)
user_replied_tweets  = extract_user_replied_tweets(interests_tweets, uid)
replied_to_user_tweets  = extract_replied_to_user_tweets(interests_tweets, uid)
quoted_user_tweets  = extract_quoted_user_tweets(interests_tweets, uid)
retweeted_user_tweets  = extract_retweeted_user_tweets(interests_tweets, uid)
explore_tweets = extract_explore_tweets(interests_tweets)


In [ ]:
print('Number of user_origin_tweets:', len(user_origin_tweets), '\n')
ts = [t['text'].replace('\u200c', '').split('\n') for t in user_origin_tweets]
for t in ts:
    print('-'*100)
    for line in t:
        print(line)


Number of user_origin_tweets: 1 

----------------------------------------------------------------------------------------------------
تصویری از جنایات جمهوری اسلامی ایران در آبان ۹۸ 
در همان روزهای اعتراضات تعدادی از جوانان ناپدید شدند و پس از چند روز اجساد آنان دست و پا بسته همراه با آثار شکنجه بر بدن در سدهای مریوان ،سنندج ،خوزستان وکرج پیدا شدند . اسامی تعدادی از این جانباختگان به شرح زیر است:
عرفان ساعدپناه 
ارشاد رحمانیان 
هیوا رحیمی 
مسعود امینی 
رضا صادقی
سوران محمدی 
علی جواهری 
#نه_میبخشیم_نه_فراموش_میکنیم 
#آبان_۹۸


In [ ]:
print('Number of user_quoted_tweets:', len(user_quoted_tweets), '\n')
ts = [t['text'].replace('\u200c', '').split('\n') for t in user_quoted_tweets]
for t in ts:
    print('-'*100)
    for line in t:
        print(line)


Number of user_quoted_tweets: 3 

----------------------------------------------------------------------------------------------------
بجای درست کردن «حزب»های کارتونی برای کردستان، بروید کمی از مبارزه کردها، مقاومتشان، سازماندهیشان، شهامتشان و مبارزات سکولار، جدی و استخواندار آنها یاد بگیرید. شما تنها کاری که میتوانید بکنید این است که به سخنرانی مانیفستوار مادران و پدران کردستان بر مزار عزیزانشان، گوش بدهید. و کمی فرهنگ سیاسی، کمی مبارزه، کمی شهامت و کمی آزادگی بیاموزید. کردستان، توییتر فارسی نیست.
#ژن_ژیان_ئازادی 
#ژینا_امینی
----------------------------------------------------------------------------------------------------
جمهوری اسلامی پیشمرگ مسلمان کرد درست کرد.
رضا پهلوی فرمان تاسیس حزب کردی داد.
در هر حال برای ما هر دو گروه جاش هستند. https://t.co/WautxEriQF
----------------------------------------------------------------------------------------------------
بجای درست کردن «حزب»های کارتونی برای کردستان، بروید کمی از مبارزه کردها، مقاومتشان، سازماندهیشان، شهامتشان و مبارزات سکولار

In [ ]:
print('Number of user_retweeted_tweets:', len(user_retweeted_tweets), '\n')
ts = [t['text'].replace('\u200c', '').split('\n') for t in user_retweeted_tweets]
for t in ts:
    print('-'*100)
    for line in t:
        print(line)


Number of user_retweeted_tweets: 4 

----------------------------------------------------------------------------------------------------
سوزش #فرقه_پهلوی بخاطر رقص گلشیفته  نیست
بخاطر این تصویر و همدلی بین این افراده!
 رسانه های سپاه را هم نگاه کنید و با رفتار این فرقه مقایسه کنید!
هیچ چیز اتفاقی نیست
هیچ چیز عادی نیست!
یک روز #روح_الله_زم  را تخریب کردند
یک روز...
#مهسا_امينی https://t.co/XZ8ZRvoDEs
----------------------------------------------------------------------------------------------------
شبهات قانونی حکم اعدام #محسن_شکاری 

اجرای حکم اعدام ،تنها ۱۸ روز پس از صدور ،موجی از واکنشها و محکومیت را در پی داشت.در اینکه اعدام او قانونی بود یا خیر باید گفت چون اتهام وی بستن خیابان ستارخان و مجروح کردن یک عضو نیروی بسیج بود، رای هم شبهه موضوعی دارد و هم شبهه حکمی!؟ ۱
----------------------------------------------------------------------------------------------------
جنایاتی که مرتکب شدەاید، خونهایی که ریختە و قصاوتی که داشتەاید را هزاران بار دیگر یادآوری میکنیم. به همه مردمان جهان ن

In [ ]:
print('Number of user_replied_tweets:', len(user_replied_tweets), '\n')
ts = [t['text'].replace('\u200c', '').split('\n') for t in user_replied_tweets]
for t in ts:
    print('-'*100)
    for line in t:
        print(line)


Number of user_replied_tweets: 6 

----------------------------------------------------------------------------------------------------
@KeyOne367 عین شبکه من و تو از قبل همه رو انتخاب کردند
----------------------------------------------------------------------------------------------------
@fariba312 @SGhasseminejad انقدر سرشون تو اخور پهلویه نمیدونن چی به چیه 😁
----------------------------------------------------------------------------------------------------
@KambizGhafouri @fariba312 خوب شد رضا شاه مرد و این روزها زو ندید وگرنه رضا پهلوی رو درجا خنثی میکرد
----------------------------------------------------------------------------------------------------
@2Ubbe برای گذار از جمهوری اسلامی با احدی تعارف نداریم ولی به نظرم این متن نوشته شده بیشتر از روی احساسات توسط یک سری از خانواده های دادخواه پخش شد و نمیدانستند اصل داستان از چه سمتی است
----------------------------------------------------------------------------------------------------
@joker_yaghi67 کی فکرش رو میکرد عاقبت سلطنت

In [ ]:
print('Number of explore_tweets: ', len(explore_tweets), '\n')
ts = [t['text'].replace('\u200c', '').split('\n') for t in explore_tweets]
for t in ts:
    print('-'*100)
    for line in t:
        print('\t',line)


Number of explore_tweets:  3 

----------------------------------------------------------------------------------------------------
	 ۲۱ آذر ۱۳۳۷ (۱۲ سال پس از #نجات_آذربایجان از هاضمه شوروی) دو رژه در تهران و تبریز بنام "جشن ارتش ایران"  برگزار شد. اینها از نخستین قدم هایی بود که ارتش نو نَوا شده ایران بر روی زمین می کوبید.
	  میراث رضاشاه که با آن شیخ خزعل و سیمیتقو  را درهم کوبیده بود، با قطع ید ریزه خواران استالین در آذربایجان و مهاباد دوباره جان گرفت، سپس برای کمک به پادشاه عمان به جنگ دست نشاندگان شوروی در ظفار رفت و بعد مهمترین شریک ایالات متحده در پروژه های اطلاعاتی آیبکس و ژن تیره شد.
	 از میانه دهه ۷۰، این ارتش جوان اما مدرن و آموزش دیده، پنجمین قدرت نظامی دنیا بود که پس از سالها مناقشه به توهمات صدام حسین بر سر اروند رود و خوزستان پایان داد.
	 راهی کوتاه، اما سخت و طاقت فرسا در محاصره گرگ ها و کفتاران و خائنان طی شد، تا ارتش نوپای رضاشاه به آنجا رسید!
----------------------------------------------------------------------------------------------------
	 من این کلیپ رو برای تب

In [ ]:
endt = datetime.datetime.now()
print(endt)


2024-01-31 23:21:17.127248


# User Recommender 

### Define the functions required to perform Min-Hashing and LSH

In [ ]:
def my_hash(x, i, N):
    '''
        random_a and random_b are a list of random integers
        p is a prime number that bigger than 32^7-1
    '''
    return ((my_hash.random_a[i] * x + my_hash.random_b[i]) % my_hash.p) % N 


def signature(tweet_list):
    '''
    This function performs Min-Hashing on tweet_list like 
    One-pass implementation which is mentioned in the slides
    and return signature vector (as a list)
    M is length of the vector that equals to r*b
    BTI is the Biggest Tweet ID 
    '''
    sig = [signature.BTI+1 for _ in range(signature.M)]
    
    for j in tweet_list:
        for i in range(signature.M):
            k = my_hash(j, i, signature.BTI)
            if k < sig[i]:
                sig[i] = k
    return sig


# It is not used in this script
def bands_hashing_lossless(x): 
    '''This function hashes r numbers into one big number without losing data'''
    out = 0
    for i in range(int(lsh.r)):
        out *= bands_hashing.NBOR
        out += x[i] % bands_hashing.NBOR
    return out


def bands_hashing(x): 
    '''This function hashes r numbers into sum of them (less than NBOR)'''
    out = 0
    for i in range(int(lsh.r)):
        out += x[i]
    return out % bands_hashing.NBOR


def lsh(sig):
    '''This function drops each signature(sig) into b buckets'''
    return [bands_hashing(sig[i*lsh.r : (i+1)*lsh.r]) for i in range(lsh.b)]



### Define parmeters

In [ ]:
signature.BTI = 100000000  # 2000000000000000000  # BTI is the Biggest Tweet ID 
bands_hashing.NBOR = signature.BTI  # Number of Bucket for One Row

r = 3  # Number of rows in Signature matrix
b = 10  # Number of bands in Signature matrix
M = r * b  # Signature size for each news
lsh.r = r
lsh.b = b
signature.M = M
print(f'Divide signature matrix into <{b}> bands of <{r}> rows')


random.seed(42)
my_hash.random_a = [random.randint(100, 400) for i in range(M)]
my_hash.random_b = [random.randint(1, 200) for i in range(M)]
my_hash.p = 104677207  # 2015121110987654321
print('\nlist of random integers for a and b in my hashing formula(ax+b):')
print('a: ', my_hash.random_a)
print('\nb: ', my_hash.random_b)


Divide signature matrix into <10> bands of <3> rows

list of random integers for a and b in my hashing formula(ax+b):
a:  [157, 112, 240, 225, 214, 171, 152, 379, 144, 316, 116, 115, 147, 211, 219, 358, 113, 387, 201, 379, 314, 212, 329, 242, 103, 181, 316, 274, 242, 179]

b:  [56, 196, 87, 27, 24, 98, 25, 92, 89, 155, 68, 12, 187, 118, 138, 32, 97, 21, 142, 76, 161, 159, 93, 148, 50, 181, 18, 12, 170, 59]


In [ ]:
# Retrieving all tweets that each user is interested in as a list (bag of tweets)
# user_interests_rdd: [(u1, [(tid: interest), ...])]
user_tweets_bag_rdd = user_interests_rdd.filter(lambda x: x[1])\
    .map(lambda x: (x[0], [int(tid) % signature.BTI for tid in x[1].keys()]))  # [(u1, [tid, ...])]

# Retrieving the users of every buckets
bucket_user_rdd = user_tweets_bag_rdd.flatMap(lambda x: [(bucket, {x[0]}) for bucket in lsh(signature(x[1]))])  # (bucket, uid)

bucket_users_rdd = bucket_user_rdd.reduceByKey(lambda x, y: x.union(y))  # (bucket, uids)

# [(bucket, {users})]  # a list of b bucket
bucket_candidate_uids = bucket_users_rdd.filter(lambda x: uid in x[1]).take(lsh.b)


In [ ]:
bucket_candidate_uids = bucket_users_rdd.filter(lambda x: uid in x[1]).take(lsh.b)
bucket_candidate_uids

In [ ]:
bucket_candidate_uids

[(21052469, {'1425068264862978049'}),
 (9161032,
  {'1348410757298458624',
   '1425068264862978049',
   '1445120935141466112',
   '1572861549991104512',
   '1601767356837117952',
   '918911124254416901'}),
 (11292176, {'1425068264862978049'}),
 (1700380,
  {'1341775133032198145',
   '1348410757298458624',
   '1425068264862978049',
   '1445120935141466112',
   '1572861549991104512',
   '1601767356837117952',
   '917647599762632704',
   '918911124254416901'}),
 (47937934,
  {'1425068264862978049', '1445120935141466112', '918911124254416901'}),
 (29465416,
  {'1425068264862978049', '1445120935141466112', '918911124254416901'}),
 (9570192,
  {'1425068264862978049', '1445120935141466112', '918911124254416901'}),
 (16193730,
  {'1425068264862978049', '1445120935141466112', '918911124254416901'}),
 (3354056,
  {'1425068264862978049',
   '1445120935141466112',
   '1643014199814025217',
   '918911124254416901',
   '989887176803528704'}),
 (20684908,
  {'1425068264862978049', '144512093514146611

In [ ]:
candidate_users = set()
for _, uids in bucket_candidate_uids:
    candidate_users = candidate_users.union(uids)

candidate_users

{'1341775133032198145',
 '1348410757298458624',
 '1425068264862978049',
 '1445120935141466112',
 '1572861549991104512',
 '1601767356837117952',
 '1643014199814025217',
 '917647599762632704',
 '918911124254416901',
 '989887176803528704'}

### Define the functions required to show similar condidate and similar news

In [ ]:
def getSimilars(cus, q_uid, threshold=0.4):
    ''' 
    This function get shingles of candidate news, calculate similarity
    of candidate news with query news and it returns news whose ratio 
    of common shingles with query news to the union of their shingles is 
    greater than threshold.

    cus: candidate users' shingles, a list of (uid, [shingle_id])
    q_uid: uid of query user
    '''
    q_shingles = []
    for i in range(len(cus)):
        if q_uid == cus[i][0]:
            q_shingles = list(cus[i][1])
            break
        
    similars = []
    for candidate in cus:
        # s is Jaccard similarity of news = (query_news ∩ candidate_news)/union(query_news U candidate_news)
        s = len(set(candidate[1])&set(q_shingles))/len(set(candidate[1])|set(q_shingles))
        if s >= threshold:    
            similars.append((candidate[0], s)) # a list of (uid, percent)

    return similars


In [ ]:
"""
you must choose user_id (uid) in this section.
you can change # in 'user_buckets[#][0]' for this purpose.
"""
q_uid = uid

# Retrieving all tweets that each user is interested in as a list (bag of tweets)
# candidate_user_shingles: a list of (uid, [shingle_id])
candidate_user_shingles = user_tweets_bag_rdd.filter(lambda x: x[0] in candidate_users)\
                                             .collect()

# similar: a list of (uid, percent of similarity)
similar = getSimilars(candidate_user_shingles, q_uid)


In [ ]:
similar_uids, similarites = zip(*similar)

interests_tweets = tweets_rdd.filter(lambda x: json.loads(x)['user']['id'] in similar_uids)\
                             .map(lambda x: dict(filter(lambda y: y[0] in necessary_keys, json.loads(x).items())))\
                             .collect()
interests_tweets2 = interests_tweets.copy()


In [ ]:
"""
you must select id of similar user (similar_uid) in this section.
you can change # in 'similar_uids[#]' for this purpose.
"""
similar_uid = similar_uids[8]


IndexError: tuple index out of range

In [ ]:
interests_tweets = interests_tweets2.copy()
user_origin_tweets = extract_user_origin_tweets(interests_tweets, similar_uid, rem=False)
user_quoted_tweets  = extract_user_quoted_tweets(interests_tweets, similar_uid, rem=False)
user_retweeted_tweets  = extract_user_retweeted_tweets(interests_tweets, similar_uid, rem=False)
user_replied_tweets  = extract_user_replied_tweets(interests_tweets, similar_uid, rem=False)


In [ ]:
print('Number of user_origin_tweets:', len(user_origin_tweets), '\n')
ts = [t['text'].replace('\u200c', '').split('\n') for t in user_origin_tweets]
for t in ts:
    print('-'*100)
    for line in t:
        print(line)


Number of user_origin_tweets: 0 



In [ ]:
print('Number of user_quoted_tweets:', len(user_quoted_tweets), '\n')
ts = [t['text'].replace('\u200c', '').split('\n') for t in user_quoted_tweets]
for t in ts:
    print('-'*100)
    for line in t:
        print(line)


Number of user_quoted_tweets: 1 

----------------------------------------------------------------------------------------------------
نرگس محمدی مثالی از «استقامت سازنده» است زیرا نه به تقیه و تسلیم تن میدهد و نه همچون بعضی مخالفانِ ج.ا از همان «سیاست کثیفِ» ستمگران حاکم (خودمحقپنداریِ جزماندیشانه،خشونتستایی،نارواداری،انحصارطلبی،دروغگویی و تهمتپراکنی) پیروی میکند.او قیدوبندهای اخلاقی را به رسمیت میشناسد


In [ ]:
print('Number of user_retweeted_tweets:', len(user_retweeted_tweets), '\n')
ts = [t['text'].replace('\u200c', '').split('\n') for t in user_retweeted_tweets]
for t in ts:
    print('-'*100)
    for line in t:
        print(line)


Number of user_retweeted_tweets: 1 

----------------------------------------------------------------------------------------------------
بجای درست کردن «حزب»های کارتونی برای کردستان، بروید کمی از مبارزه کردها، مقاومتشان، سازماندهیشان، شهامتشان و مبارزات سکولار، جدی و استخواندار آنها یاد بگیرید. شما تنها کاری که میتوانید بکنید این است که به سخنرانی مانیفستوار مادران و پدران کردستان بر مزار عزیزانشان، گوش بدهید. و کمی فرهنگ سیاسی، کمی مبارزه، کمی شهامت و کمی آزادگی بیاموزید. کردستان، توییتر فارسی نیست.


In [ ]:
print('Number of user_replied_tweets:', len(user_replied_tweets), '\n')
ts = [t['text'].replace('\u200c', '').split('\n') for t in user_replied_tweets]
for t in ts:
    print('-'*100)
    for line in t:
        print(line)


Number of user_replied_tweets: 4 

----------------------------------------------------------------------------------------------------
@aghferiii @andfogle سال 1302، مدرس رو به رضا خان سردار سپه وقت:
"...پس چرا باید طوری صحبت کنی که آدم یاد بسیجی های چاق و احمق بیوفته؟" https://t.co/HDMUuVBqQm
----------------------------------------------------------------------------------------------------
@_essisajjad چطور نیستی وقتی سَرت اینطور تو کون دیگرانه؟!
----------------------------------------------------------------------------------------------------
@hamshahrinews چه بازی تکراریی. ج.ا تا میبینه در مقابله مستقیم ضعف داره و حمله مستقیمش کاری نیست، سریع تمرکزش رو میبره روی اینکه به گروه های مخالفش بگه "نگاه کنید چقدر رقیب همین، نگاه اَاا چطور این اون رو ناک اوت کرد، نگاه چه شکاف عمیقی بینتون هست، چقــــــــــــــدر اختلاف دارین شماها، پشمام!"
----------------------------------------------------------------------------------------------------
@Me_Unstoppable2 یعنی اگه آمریکا برای یک جایزه 

In [ ]:
# sys.getsizeof(tweets)
sys.getsizeof(explore_tweets)


120